In [15]:
import numpy as np
import em_algorithm
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal
import models
import em_algorithm
import pandas as pd
import time
from girth import multidimensional_twopl_mml
from girth import twopl_mml
import cma

In [16]:
def params_to_vector(model):
    item_dimension = model.item_dimension
    latent_dimension = model.latent_dimension
    A_flat = model.item_parameters["discrimination_matrix"].flatten()
    A_flat = A_flat[A_flat != 0]
    A_cut = len(A_flat)
    delta_cut = A_cut+item_dimension
    delta = model.item_parameters["intercept_vector"]
    sigma_flat = model.person_parameters["covariance"][np.triu_indices(
            model.latent_dimension, k=1)]
    return(np.concatenate((A_flat, delta, sigma_flat), axis=0))

def vector_to_params(vector, model):
    item_dimension = model.item_dimension
    latent_dimension = model.latent_dimension 
    q_matrix = model.item_parameters["q_matrix"] 
    q_flat = q_matrix.flatten()
    A_cut = len(q_flat[q_flat != 0])
    delta_cut = A_cut+item_dimension
    A = vector[0:A_cut]
    A = model.fill_zero_discriminations(A).reshape((item_dimension, latent_dimension))
    delta = vector[A_cut:delta_cut]
    item_parameters = {"discrimination_matrix": A, "intercept_vector": delta}
    corr = vector[delta_cut: len(vector)]
    sigma = model.corr_to_sigma(corr, False)
    return({"item_parameters": item_parameters, "person_parameters": {"covariance": sigma}})

def direct_marginal_optimization(model, response_data):
    #Get initial parameters
    x0 = params_to_vector(model)
    response_data = response_data.to_numpy()
    #Optimize with GA
    def marginal_l_func(input_vector):
        parameters = vector_to_params(input_vector, model)
        try:
            model.set_parameters(parameters)
        except Exception:
            return(np.inf)
        result = -1*model.marginal_response_loglikelihood(response_data)
        return(result)
    es = cma.CMAEvolutionStrategy(x0=x0, sigma0=0.5)
    es.optimize(marginal_l_func, maxfun=100000)
    #Get result
    result = es.result.xfavorite
    return(vector_to_params(result, model))

In [23]:
from models.mirt_2pl import mirt_2pl


def rmse(y_pred: np.array, y_true: np.array) -> float:
    MSE = np.square(np.subtract(y_pred.flatten(), y_true.flatten())).mean()
    RMSE = np.sqrt(MSE)
    return(float(RMSE))

def experiment_performance(estimated_parameter_dict, real_parameter_dict):
    res_dict = {}
    if "item_parameters" in estimated_parameter_dict.keys():
        A_pred = estimated_parameter_dict["item_parameters"]["discrimination_matrix"]
        delta_pred = estimated_parameter_dict["item_parameters"]["intercept_vector"]

        A_true = real_parameter_dict["item_parameters"]["discrimination_matrix"]
        delta_true = real_parameter_dict["item_parameters"]["intercept_vector"]

        print("Absolute diff in A:")
        print(np.abs(A_true-A_pred))

        print("Absolute diff in delta:")
        print(np.abs(delta_true-delta_pred))

        res_dict["rmse_A"] = rmse(A_pred, A_true)
        res_dict["rmse_delta"] = rmse(delta_true, delta_pred)

    if "person_parameters" in estimated_parameter_dict.keys():
        sigma_pred = estimated_parameter_dict["person_parameters"]["covariance"]
        sigma_true = real_parameter_dict["person_parameters"]["covariance"]

        print("Absolute diff in sigma:")
        print(np.abs(sigma_true-sigma_pred)) 

        res_dict["rmse_sigma"] = rmse(sigma_true, sigma_pred) 
    if len(res_dict.keys())==0:
        raise Exception("No performance to calculate")
    return(res_dict)

def create_parameter_dict(estimated_early_parameters, real_early_parameters, estimated_late_parameters, real_late_parameters):
    parameter_dict = {"real_early_parameters": real_early_parameters,
                   "estimated_early_parameters": estimated_early_parameters}
    return(parameter_dict)


def create_performance_dict(parameter_dict, run_dict, sample=None, baselines=None, model=None):
    result_dict = parameter_dict
    result_dict["sample"] = sample
    #Model Performance
    result_dict["early_performance"] = {}
    result_dict["early_performance"]["rmse"] = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=result_dict["estimated_early_parameters"])
    #Baseline's Performance
    baselines["early_initial"]["performance"] = {"rmse": experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=baselines["early_initial"]["parameters"])}
    if "girth" in baselines.keys():
        girth_data = result_dict["sample"]["early_responses"].to_numpy().transpose()
        if model.latent_dimension == 1:
            girth_estimates = twopl_mml(girth_data)
        else:
            girth_estimates = multidimensional_twopl_mml(girth_data, n_factors=model.latent_dimension)
        girth_item_parameters = {"discrimination_matrix": girth_estimates["Discrimination"], "intercept_vector": girth_estimates["Difficulty"]}
        girth_parameters = {"item_parameters": girth_item_parameters}
        girth_performance_rmse = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=girth_parameters)
        baselines["girth"]["parameters"] = girth_parameters
        baselines["girth"]["performance"] = {"rmse": girth_performance_rmse}

    if "direct_early" in baselines.keys():
        dir_model = mirt_2pl(item_dimension=model.item_dimension, latent_dimension=model.latent_dimension, Q=model.item_parameters["q_matrix"])
        direct_early_item_parameters = direct_marginal_optimization(dir_model, response_data=sample["early_responses"])
        direct_early_performance_rmse = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=direct_early_item_parameters)
        baselines["direct_early_marginal"]["parameters"] = direct_early_item_parameters
        baselines["direct_early_marginal"]["performance"] = {"rmse": direct_early_performance_rmse}

    result_dict["baselines"] = baselines
    likelihood = calculate_marginal_likelihoods(model=model, response_data=sample["early_responses"], real_parameters=result_dict["real_early_parameters"],
                                                initial_parameters=baselines["early_initial"]["parameters"], estimated_parameters=result_dict["estimated_early_parameters"])
    result_dict["early_performance"]["marginal_likelihood"] = likelihood
    result_dict["early_performance"]["run"] = run_dict["early"]
    return(result_dict)

def calculate_marginal_likelihoods(model, response_data, real_parameters, initial_parameters, estimated_parameters):
    model.set_parameters(initial_parameters)
    initial_marginal_likelihood = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    model.set_parameters(real_parameters)
    optimal_marginal_likelihood = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    model.set_parameters(estimated_parameters)
    marginal_likelihood_estimated = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    likelihood_dict = {"optimal": optimal_marginal_likelihood, "estimated": marginal_likelihood_estimated, "initial": initial_marginal_likelihood}
    return(likelihood_dict)

In [24]:
def direct_optimization_benchmark():
    latent_dimension = 2
    item_dimension = 6
    sample_size=30

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension = item_dimension, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(100)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, Q=Q)
    model.initialize_from_responses(response_data=sample["early_responses"])

    estimated_params = direct_marginal_optimization(model, response_data=sample["early_responses"])
    return(estimated_params)

In [25]:
#direct_result_dict = direct_optimization_benchmark()

## Experiment 0: MIRT-2PL Performance Benchmark

In [26]:
def mirt_performance_benchmark() -> dict:
    latent_dimension = 2
    item_dimension = 6
    sample_size=30

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension = item_dimension, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(100)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, Q=Q)
    model.initialize_from_responses(response_data=sample["early_responses"])
    initial_early_parameters = model.get_parameters()
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=50)
    run_time =  (time.time() - start_time)

    #Create Baselines
    baselines = {"early_initial": {"parameters": initial_early_parameters}, "girth": {}}
    
    #Create results
    early_estimated_parameters = em.model.get_parameters()

    parameter_dict = create_parameter_dict(estimated_early_parameters=early_estimated_parameters,
                          real_early_parameters = real_early_parameters,
                          estimated_late_parameters=None, real_late_parameters=None)
    run_dict = {"early": {"runtime": run_time,
                "number_steps": em.n_steps}}
    performance_dict = create_performance_dict(parameter_dict=parameter_dict, run_dict=run_dict, sample=sample, baselines=baselines, model=model)
    return(performance_dict)

In [27]:

def print_result(result_dict, description=""):
    #Description
    print("------------------------------------")
    print("##### Results for {0}".format(description))
    #Performance/time
    ep_dict = result_dict["early_performance"]
    runtime = np.round(ep_dict["run"]["runtime"], 2)
    steps = ep_dict["run"]["number_steps"]
    print("Runtime: {0} seconds, {1} steps, {2} seconds per step \\".format(runtime, steps, np.round(runtime/steps, 2)))
    #Performance/results
    l_optimal = np.round(ep_dict["marginal_likelihood"]["optimal"], 2)
    l_estimated = np.round(ep_dict["marginal_likelihood"]["estimated"], 2)
    l_real = np.round(ep_dict["marginal_likelihood"]["initial"], 2)
    print("Optimal marginal Likelihood: {0}, Estimated: {1}, Initial {2}".format(l_optimal,l_estimated,l_real))
    #print("Performance: rmse-mean = {0} \\".format(np.round(np.mean(np.array(list(result_dict["early_performance"].values()))), 4)))
    rmse_model = ep_dict["rmse"]
    rmse_girth = result_dict["baselines"]["girth"]["performance"]["rmse"]
    rmse_initial = result_dict["baselines"]["early_initial"]["performance"]["rmse"]
    rmse_frame = pd.DataFrame(columns=["rmse_A", "rmse_delta", "rmse_sigma"])
    rmse_frame = rmse_frame.append(rmse_model, ignore_index=True)
    rmse_frame = rmse_frame.append(rmse_girth, ignore_index=True)
    rmse_frame = rmse_frame.append(rmse_initial, ignore_index=True)
    rmse_frame.index = ["Estimated", "Girth", "Initial"]
    print(rmse_frame.to_markdown())

                                                                                

In [6]:
performance_dict = mirt_performance_benchmark()

EM Iteration 1
E-step
M-step
Maximize Q-0


c:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\em_algorithm\e_step_mirt_2pl.py:90: RuntimeWarning: divide by zero encountered in log
  factor = np.log(self.model.latent_density(theta, sigma=sigma))


Maximize the Q_i's
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=1067195, Mon Sep 12 17:18:50 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 8.125752963487031e+01 1.0e+00 5.07e-01  4e-01  5e-01 0:00.0
    2     12 6.952096045967318e+01 1.2e+00 6.97e-01  7e-01  8e-01 0:00.1
    3     18 6.385417860928425e+01 1.3e+00 9.14e-01  1e+00  1e+00 0:00.1
  100    600 6.299140930836132e+01 7.7e+00 2.75e-01  2e-02  2e-02 0:02.9
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=1115007, Mon Sep 12 17:18:55 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 6.636697273628047e+01 1.0e+00 5.56e-01  5e-01  6e-01 0:00.0
    2     12 6.608464583060804e+01 1.4e+00 4.70e-01  4e-01  5e-01 0:00.1
    3     18 6.368038838306229e+01 1.5e+00 5.17e-01  4e-01  5e-01 0:00.1
  100    600 6.272959609677451e+01 8.8e+00 2.44e-03  1e-05  3e-05 0:02.9
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=1006819, Mon Se

In [7]:
print_result(performance_dict, "pycma ga used")

NameError: name 'print_result' is not defined

------------------------------------
##### Results for Naive, all-Genetic, no vectorization
Runtime: 29100 seconds, 100+ steps, 291 seconds per step\
Performance: rmse-mean = 0.5896464054719196

------------------------------------
##### Results for Vectorized q_item
Runtime: 613.3871161937714 seconds, 100+ steps, 6.0731397642947655 seconds per step \
Performance: rmse-mean = 0.6466789259215839

------------------------------------
##### Results for More precise results through higher N and pop_size
Runtime: 2239.073846578598 seconds, 100+ steps, 22.169047985926714 seconds per step \
Performance: rmse-mean = 0.6384660709857508

------------------------------------
##### Results for Debug GA sorting, Add Likelihood-based stopping criterion
Runtime: 135.11203932762146 seconds, 7 steps, 19.301719903945923 seconds per step \
Performance: rmse-mean = 0.5616448369465917

------------------------------------
##### Results for Add vectorization in Q_0
Runtime: 228.78 seconds, 17 steps, 13.46 seconds per step \
Performance: rmse-mean = 0.6174

------------------------------------
##### Results for Add vectorization in E-Step normalizing constant
Runtime: 28.37 seconds, 9 steps, 3.15 seconds per step \
Performance: rmse-mean = 0.5808

------------------------------------
##### Results for Add Suggested Initial Parameters (Zhang)
Runtime: 61.22 seconds, 19 steps, 3.22 seconds per step \
Performance: rmse-mean = 0.5625

------------------------------------
##### Results for Fixed Q_item
Runtime: 81.11 seconds, 12 steps, 6.76 seconds per step \
Optimal marginal Likelihood: -375.43, Estimated: -393.8, Initial -391.83
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.762005 |     0.512703 |     0.600404 |
| Girth     | 1.61804  |     0.750525 |   nan        |
| Initial   | 0.770281 |     0.227528 |     0.141421 |

------------------------------------
##### Results for Inproved GA stoppimg rule
Runtime: 58.99 seconds, 11 steps, 5.36 seconds per step \
Optimal marginal Likelihood: -362.48, Estimated: -385.24, Initial -368.28
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.888581 |     0.485861 |    0.0122515 |
| Girth     | 0.943352 |     0.359848 |  nan         |
| Initial   | 0.509902 |     0.236338 |    0.141421  |

------------------------------------
##### Results for pycma ga used
Runtime: 489.8 seconds, 16 steps, 30.61 seconds per step \
Optimal marginal Likelihood: -381.34, Estimated: -414.28, Initial -379.77
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.942127 |     0.505808 |     0.215139 |
| Girth     | 1.42144  |     0.632431 |   nan        |
| Initial   | 0.509902 |     0.489947 |     0.141421 |

## Procedure for better runtime

1. Add Performance Benchmark (done)
2. Add Vectorization to q_item (done)
3. Add better Initialization (done)
3. Add Vectorization to q_0 (done)
4. Add q_0 Derivation 1. + BFGS
5. Add q_0 Derivation 2. + Newton Raphson
6. Use cython (maybe)

## Experiment 1: MIRT-2PL Parameter Recovery

In [29]:
from unittest import result


def mirt_param_recovery(sample_size, item_dimension = 20, latent_dimension=3, q_type="seperated") -> dict:
    
    #Define Population
    population = respondent_population(latent_dimension=latent_dimension)
    real_latent_cov = population.initialize_random_person_parameters()

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension=item_dimension)
    response_simulation_obj.initialize_random_q_structured_matrix(structure=q_type)
    early_item_parameters = response_simulation_obj.initialize_random_item_parameters()
    early_item_parameters.update({"item_dimension": item_dimension, "latent_dimension": latent_dimension})
    sample = response_simulation_obj.sample(sample_size=sample_size)
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=latent_dimension, item_dimension=item_dimension, Q=early_item_parameters["q_matrix"])
    model.initialize_from_responses(response_data=sample["early_responses"])
    initial_early_parameters = model.get_parameters()
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=50)
    run_time =  (time.time() - start_time)

    #Measure Performance
    early_estimated_item_parameters = em.model.item_parameters
    early_estimated_person_parameters = em.model.person_parameters

    #Create Baselines
    baselines = {"early_initial": {"parameters": initial_early_parameters}, "girth": {}, "direct_early": {}}
    
    #Create results
    early_estimated_parameters = em.model.get_parameters()

    parameter_dict = create_parameter_dict(estimated_early_parameters=early_estimated_parameters,
                          real_early_parameters = real_early_parameters,
                          estimated_late_parameters=None, real_late_parameters=None)
    run_dict = {"early": {"runtime": run_time,
                "number_steps": em.n_steps}}
    performance_dict = create_performance_dict(parameter_dict=parameter_dict, run_dict=run_dict, sample=sample, baselines=baselines, model=model)

    return(performance_dict)

In [30]:
#Idee, Delta so initialisieren, dass die relativen Itemschwierigkeiten um 0 herum verteilt sind!
result_dict = mirt_param_recovery(200, q_type="singular", item_dimension=6, latent_dimension=1)

EM Iteration 1
E-step
M-step
Maximize Q-0
Maximize the Q_i's
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=90025, Mon Sep 12 19:03:55 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 9.434801830170177e+01 1.0e+00 4.15e-01  4e-01  4e-01 0:00.1
    2     12 9.297205294913094e+01 1.1e+00 4.41e-01  4e-01  4e-01 0:00.1
    3     18 9.336096174651361e+01 1.1e+00 3.48e-01  2e-01  3e-01 0:00.2
   59    354 9.180083164441116e+01 1.5e+00 2.15e-01  4e-02  4e-02 0:03.2
  100    600 9.205427950085708e+01 5.3e+00 1.03e-01  4e-03  1e-02 0:05.4
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=144800, Mon Sep 12 19:04:04 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 9.292933833705320e+01 1.0e+00 3.74e-01  3e-01  4e-01 0:00.1
    2     12 9.474889007817364e+01 1.3e+00 3.66e-01  3e-01  4e-01 0:00.1
    3     18 9.207424072962004e+01 1.5e+00 3.68e-01  3e-01  3e-01 0:00.2
   59    354 9.130719026160499e+

c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: sigma change np.exp(1.0184335552438248) = 2.7688541081547724 clipped to np.exp(+-1) (class=CMAAdaptSigmaCSA method=update iteration=144)
  warnings.warn(msg + ' (' +


(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=181858, Mon Sep 12 19:23:30 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 8.794578763821463e+01 1.0e+00 3.57e-01  3e-01  3e-01 0:00.1
    2     12 8.751343007861279e+01 1.2e+00 2.78e-01  2e-01  2e-01 0:00.1
    3     18 8.787876029315335e+01 1.3e+00 2.33e-01  1e-01  2e-01 0:00.2
   61    366 9.098290446204584e+01 1.2e+01 5.49e-01  4e-02  3e-01 0:03.2
  100    600 8.126653486968978e+01 1.0e+02 1.65e-01  8e-03  5e-02 0:05.2
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=20250, Mon Sep 12 19:23:40 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 8.668782467286674e+01 1.0e+00 5.65e-01  6e-01  6e-01 0:00.1
    2     12 8.665638932529531e+01 1.2e+00 4.94e-01  4e-01  5e-01 0:00.1
    3     18 8.419691260604500e+01 1.3e+00 4.75e-01  4e-01  5e-01 0:00.2
   61    366 8.693577416780758e+01 4.0e+00 2.15e-01  2e-02  4e-02 0:03.2
  100    600 7.89890

In [10]:
print_result(result_dict, "Improved Sampling of Item Parameters")

------------------------------------
##### Results for Improved Sampling of Item Parameters
Runtime: 497.94 seconds, 51 steps, 9.76 seconds per step \
Optimal marginal Likelihood: -486.98, Estimated: -1885.8, Initial -524.06
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated |  1.45421 |      6.48212 |            0 |
| Girth     |  1.09907 |      4.89271 |          nan |
| Initial   |  1.54631 |      1.42726 |            0 |


In [9]:
import cma
es = cma.CMAEvolutionStrategy(x0 = 2 * [0], sigma0 = 0.5)
es.optimize(lambda x: x[0]**2 + x[1]**2)
es.result


(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=1097407, Mon Sep 12 16:47:00 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 4.334531222846078e-02 1.0e+00 4.06e-01  4e-01  4e-01 0:00.0
    2     12 6.754399694251606e-04 1.0e+00 3.13e-01  2e-01  3e-01 0:00.0
    3     18 5.231249853593185e-04 1.1e+00 2.74e-01  2e-01  2e-01 0:00.0
   69    414 2.512717382269417e-18 1.2e+00 1.08e-05  1e-08  2e-08 0:00.1


CMAEvolutionStrategyResult(xbest=array([-8.22400933e-10, -1.35512881e-09]), fbest=2.5127173822694175e-18, evals_best=410, evaluations=414, iterations=69, xfavorite=array([-1.44813761e-10, -5.89912957e-09]), stds=array([1.65569538e-08, 1.33483903e-08]), stop={'tolfun': 1e-11})

In [11]:
es.result.xfavorite

array([-1.44813761e-10, -5.89912957e-09])

------------------------------------
##### Results for 
Runtime: 533.25 seconds, 67 steps, 7.96 seconds per step \
Performance: rmse-mean = 1.3004 \
Optimal marginal Likelihood: -588.1179947526506, Estimated: -680.8309156239219, Initial -615.7743489232016

------------------------------------
##### Results for Inproved sampling of Item Parameters
Runtime: 280.24 seconds, 35 steps, 8.01 seconds per step \
Performance: rmse-mean = 1.0605 \
Optimal marginal Likelihood: -514.6858001892723, Estimated: -734.1718399644417, Initial -525.370899764539

------------------------------------
##### Results for Improved GA stopping rule
Runtime: 497.94 seconds, 51 steps, 9.76 seconds per step \
Optimal marginal Likelihood: -486.98, Estimated: -1885.8, Initial -524.06
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated |  1.45421 |      6.48212 |            0 |
| Girth     |  1.09907 |      4.89271 |          nan |
| Initial   |  1.54631 |      1.42726 |            0 |

### Procedure for better model performance:

1. Implement MIRT Baseline (done)
2. Test GA with quadratic function (done)
3. use pycma (done)
4. Test mirt-functions (response-matrix-probability)
5. Read on MC MIRT
6. Use importance sampling
7. Implement Newton Raphson


### Estimeted Parameters